In [ ]:
import osmnx as ox
import geopandas as gpd
import folium
import openai
import json

USE_STREET_VIEW_TREES = False  # Set True to use street-view data instead of OSM

# If using street-view, provide a GeoDataFrame `street_view_trees` with Point geometries
# Example (replace with actual street-view tree detection results):
street_view_trees = gpd.read_file("/notebooks/street_view_trees.geojson")

if USE_STREET_VIEW_TREES:
    print("Using street-view detected trees")
    try:
        existing_trees = street_view_trees.to_crs(epsg=3857)
        existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
        print(f"Street-view trees found: {len(existing_trees)}")
    except Exception as e:
        print(f"Error loading street-view trees: {e}")
        existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])
else:
    # Fallback to OSM trees
    tree_tags = {'natural': 'tree'}
    try:
        existing_trees = ox.features_from_bbox(bbox, tree_tags)
        existing_trees = existing_trees.to_crs(epsg=3857)
        existing_trees = existing_trees[existing_trees.geometry.notnull()]
        existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
        print(f"Existing trees (OSM) found: {len(existing_trees)}")
    except Exception as e:
        print(f"Error fetching existing trees: {e}")
        existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Define TTDI bounding box
north, south, east, west = 3.1505, 3.128, 101.641, 101.615
bbox = (west, south, east, north)

# Fetch plantable areas (parks, grass, meadows)
tags = {
    'leisure': 'park',
    'landuse': 'grass',
    'natural': 'grass'
}

try:
    plantable_areas = ox.features_from_bbox(bbox, tags)
    plantable_areas = plantable_areas.to_crs(epsg=3857)
    plantable_areas = plantable_areas[plantable_areas.geometry.notnull()]
    # Explode mixed geometries
    plantable_areas = plantable_areas.explode(index_parts=False).reset_index(drop=True)
    plantable_areas = plantable_areas[plantable_areas.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    print(f"Plantable areas found: {len(plantable_areas)}")
except Exception as e:
    print(f"Error fetching plantable areas: {e}")
    plantable_areas = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Fetch existing trees from OSM
tree_tags = {'natural': 'tree'}
try:
    existing_trees = ox.features_from_bbox(bbox, tree_tags)
    existing_trees = existing_trees.to_crs(epsg=3857)
    existing_trees = existing_trees[existing_trees.geometry.notnull()]
    existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
    print(f"Existing trees found: {len(existing_trees)}")
except Exception as e:
    print(f"Error fetching existing trees: {e}")
    existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Remove areas too close to existing trees
if not existing_trees.empty and not plantable_areas.empty:
    existing_trees['geometry'] = existing_trees.geometry.buffer(2)  # 2m buffer
    plantable_areas = gpd.overlay(plantable_areas, existing_trees, how='difference')
    print(f"Plantable areas after removing existing trees: {len(plantable_areas)}")

# AI-based tree suggestion
def suggest_tree_ai(area_m2: float, context="urban Malaysia"):
    """
    Suggest a tropical tree for a plot in Malaysia using OpenAI.
    Returns a dictionary with tree_name, reason, height_m, spread_m
    """
    prompt = f"""
    You are a landscape expert. Suggest a tropical tree suitable for planting
    in {context}. The available plot is {area_m2:.1f} square meters.
    Provide:
    - Tree name
    - Reason why it is suitable for this plot size
    - Approximate height and spread (in meters)
    Format as JSON:
    {{
        "tree_name": "...",
        "reason": "...",
        "height_m": ...,
        "spread_m": ...
    }}
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        content = response.choices[0].message.content
        tree_info = json.loads(content)
    except Exception as e:
        # Fallback in case AI fails
        tree_info = {
            "tree_name": "Unknown",
            "reason": f"Fallback due to error: {e}",
            "height_m": None,
            "spread_m": None
        }
    return tree_info

# Calculate plot areas and suggest trees
plantable_areas['area_m2'] = plantable_areas.geometry.area
plantable_areas['suggested_tree'] = plantable_areas['area_m2'].apply(lambda a: suggest_tree_ai(a))

# Convert to WGS84 for Folium
plantable_areas = plantable_areas.to_crs(epsg=4326)
existing_trees = existing_trees.to_crs(epsg=4326)

center_lat = (north + south) / 2
center_lon = (east + west) / 2

# Create Folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='CartoDB positron')

# Shade plantable areas
for _, row in plantable_areas.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature, color="blue": {
            'fillColor': color,
            'color': 'darkblue',
            'weight': 1,
            'fillOpacity': 0.5
        },
        tooltip=f"Area: {row['area_m2']:.1f} m²\nSuggested tree: {row['suggested_tree']['tree_name']}"
    ).add_to(m)

# Shade existing tree areas (green)
for _, row in existing_trees.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature: {
            "fillColor": "green",
            "color": "green",
            "weight": 1,
            "fillOpacity": 0.5
        },
        tooltip="Existing tree"
    ).add_to(m)

m


ModuleNotFoundError: No module named 'openai'